In [169]:
import pandas as pd
import numpy as np
from numpy import savetxt
import matplotlib.pyplot as plt
%matplotlib inline

from numpy import asarray
from numpy import savetxt
from scipy.spatial.distance import euclidean

import nltk; nltk.download('stopwords')
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer 
nltk.download('wordnet')

import re
import string
from tqdm import tqdm
import time
import torch

from flair.embeddings import FlairEmbeddings, DocumentPoolEmbeddings,BertEmbeddings
from flair.data import Sentence

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_score 
import scipy.cluster.hierarchy as shc
from sklearn.preprocessing import StandardScaler, normalize 

import warnings
warnings.filterwarnings("ignore")

# initialise embedding classes
flair_embedding_forward = FlairEmbeddings('news-forward')
flair_embedding_backward = FlairEmbeddings('news-backward')
bert_embedding = BertEmbeddings('bert-base-uncased')

# combine word embedding models
document_embeddings = DocumentPoolEmbeddings([bert_embedding, flair_embedding_backward, flair_embedding_forward])

global stopwords
stopwords = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", 
             "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 
             'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 
             'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 
             'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 
             'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 
             'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 
             'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 
             'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 
             'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 
             'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', 
             "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', 
             "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', 
             "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 
             'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 
             'wouldn', "wouldn't"]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aprabh13\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\aprabh13\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [8]:
Base_Data=pd.read_excel(r"C:\Users\aprabh13\OneDrive - Ameriprise\NLP\Survey_report.xlsx")
Base_Data=Base_Data.drop(columns=['S.No'])
Base_Data

,What would it take to rate your satisfaction with eWorkflow as extremely satisfied?
0,.
1,3
2,3 out of 5
3,5
4,6/10
...,...
108,There are times that it goes a little crazy ca...
109,There are too many blank screens \nToo often t...
110,There is some network issue due to which I am ...
111,There would be less errors and loading time wh...


In [9]:
Base_Data.rename(columns={Base_Data.columns[0]:"Data"},inplace=True)
Base_Data.head()

,Data
0,.
1,3
2,3 out of 5
3,5
4,6/10


In [14]:
Base_Data=Base_Data.astype(str)
Base_Data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113 entries, 0 to 112
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Data    113 non-null    object
dtypes: object(1)
memory usage: 1.0+ KB


In [74]:
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    text = re.sub('[^a-zA-Z0-9]{3,}', ' ', text)
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    return text


tokenizer = RegexpTokenizer(r'\w+')

def preprocess(text):
    data =''
    for token in tokenizer.tokenize(text):
        token=token.lower()
        token=WordNetLemmatizer().lemmatize(token,pos='v')
        if token not in stopwords_data :
            data=data+(token)+' '
    return data

def Only_Tokens_350(text):
    text = text.split();
    if len(text) <= 350:
        text = ' '.join(text);
    elif len(text) > 350:
        text = ' '.join(text[:350])
    return text

def pos_tagging(text):
    tags=nltk.pos_tag(tokenizer.tokenize(text))
    noun_verbs = [word for word,pos in tags if (pos == 'NN' or pos == 'NNP' or pos == 'NNS' or pos == 'NNPS' or pos == 'VB' or pos == 'VBD' or pos == 'VBG' or pos == 'VBN'or pos == 'VBP' or pos == 'VBZ')]
    data=''
    for word in noun_verbs:
        data=data+word+' '
    return data 

#The below function will do the cleaning, and save the tensor.

In [172]:
def Data_prep_BERT(Data,Special_words="",NV_only=0):
    global stopwords_data,df
    stopwords_data=stopwords
    stopwords_data.extend(Special_words)
    Data_wo_punc=Data['Data'].map(remove_punctuations)
    Data_preprocess=Data_wo_punc.map(preprocess,stopwords_data)
    if NV_only==1:
        Data_preprocess=Data_preprocess.map(pos_tagging)
    Data_350=Data_preprocess.map(Only_Tokens_350)
    df=pd.DataFrame(Data_350)
    df=df[(df['Data']).str.len()>1]
    df['to_match']=df.index
    df=df.reset_index(drop=True)

    # set up empty tensor
    X = torch.empty(size=(len(df.index), 7168))
    # fill tensor with embeddings
    i=0
    for text in df['Data']:
        sentence = Sentence(text)
        document_embeddings.embed(sentence)
        embedding = sentence.get_embedding()
        X[i] = embedding
        i += 1
        
    X2=X.detach().numpy()
    np.save('data.npy', X2)
    return X2

In [173]:
tensor=Data_prep_BERT(Base_Data)
tensor

array([[-0.2280308 , -0.03102523,  0.5824255 , ..., -0.00147955,
        -0.00778525,  0.01775798],
       [-0.35869953,  0.00630257,  0.621122  , ..., -0.0019366 ,
        -0.00186409,  0.0162747 ],
       [-0.1624794 , -0.22057477,  0.23012756, ..., -0.00744864,
         0.00262844,  0.03379398],
       ...,
       [-0.17269365, -0.3602194 ,  0.15253687, ..., -0.00209961,
        -0.00729852,  0.02896899],
       [-0.38563746, -0.22885871,  0.4855733 , ..., -0.00176037,
         0.00255855,  0.01354098],
       [-0.19984123,  0.3155908 ,  0.00492248, ..., -0.00658028,
         0.07159548,  0.01792974]], dtype=float32)

In [174]:
def Silhoutte(X2):
    X_red=X2
    Silhoutte = []
    for cluster in range(2,12):
        ward = AgglomerativeClustering(n_clusters=cluster,
                                   affinity='euclidean',
                                   linkage='ward')
        Silhoutte.append(silhouette_score(X_red, ward.fit_predict(X_red)))

    # converting the results into a dataframe and plotting them
    global frame
    frame = pd.DataFrame({'Cluster':range(2,12), 'Silhoutte':Silhoutte})
    x=frame["Silhoutte"].max()
    y=frame[frame["Silhoutte"]==x]['Cluster'].values[0]
    return x,y

In [175]:
value,topics=Silhoutte(tensor)

In [176]:
topics

2

In [177]:
#Top terms

from sklearn.feature_extraction.text import TfidfVectorizer
def get_top_words(documents, top_n):
    vectoriser = TfidfVectorizer(ngram_range=(1, 2),
                               max_df=0.89)
    tfidf_matrix = vectoriser.fit_transform(documents)
    feature_names = vectoriser.get_feature_names()
    df_tfidf = pd.DataFrame()
    for doc in range(len(documents)):
        words = []
        scores = []
        feature_index = tfidf_matrix[doc,:].nonzero()[1]
        tfidf_scores = zip(feature_index, [tfidf_matrix[doc, x] for x in feature_index])
        for w, s in [(feature_names[i], s) for (i, s) in tfidf_scores]:
            words.append(w)
            scores.append(s)
        df_temp = pd.DataFrame(data={'word':words, 'score':scores})
        df_temp = df_temp.sort_values('score',ascending=False).head(top_n)
        df_temp['topic'] = doc
        df_tfidf = df_tfidf.append(df_temp)
    return df_tfidf

In [178]:
def Output(topics,X2):

    N_CLUSTERS=topics
    ward = AgglomerativeClustering(n_clusters=N_CLUSTERS,
                                   affinity='euclidean',
                                   linkage='ward')
    pred_ward = ward.fit_predict(X2)
    df['topic_ac']=pred_ward
    topic_docs_AC = []
    # group text into topic-documents
    for topic in range(N_CLUSTERS):
        topic_docs_AC.append(' '.join(df[df['topic_ac']==topic]['Data'].values)) #this is what was wrong
    # apply function
    df_ac = get_top_words(topic_docs_AC, 10)
    return df['topic_ac'],df_ac

In [179]:
prediction,output=Output(2,tensor)

In [180]:
prediction.value_counts()

1    83
0    22
Name: topic_ac, dtype: int64

In [181]:
output

,word,score,topic
2,somewhat satisfy,0.301511,0
32,somewhat,0.301511,0
0,satisfy yes,0.150756,0
28,cant consistency,0.150756,0
22,good experience,0.150756,0
23,applications good,0.150756,0
24,performance applications,0.150756,0
25,already extremely,0.150756,0
26,satisfy already,0.150756,0
27,consistency extremely,0.150756,0


In [194]:
def top_sentences(N_topics,tensor):
    global df,df2,df_distance
    df2=pd.merge(Base_Data,df[["to_match","topic_ac"]],how="right",left_index=True,right_on='to_match')
    df2=df2.drop(columns=["to_match"])
    global topic_centroids
    topic_centroids = []
    for topic in range(N_topics):
        X_topic = tensor[df2.index[df2['topic_ac']==topic]]
        X_mean = np.mean(X_topic, axis=0)
        topic_centroids.append(X_mean)
 

    topic_distances = []
    for row in df2.index:
        topic_centroid = topic_centroids[df2.iloc[row]['topic_ac']]
        X_row = tensor[row]
        topic_distance = euclidean(topic_centroid, X_row)
        topic_distances.append(topic_distance)

    df2['topic_distance'] = topic_distances
    
    df_distance= df2.groupby(["topic_ac"]).apply(lambda x: x.sort_values(["topic_distance"])).reset_index(drop=True)
    df_top=df_distance.groupby('topic_ac').head(5)

    return df_top

In [195]:
Top_sentences=top_sentences(2,tensor)
Top_sentences

,Data,topic_ac,topic_distance
0,extremely satisfied,0,18.081373
1,Somewhat satisfied,0,18.251278
2,Somewhat satisfied.,0,18.251278
3,I am already extremely satisfied,0,18.321226
4,Increase in stability and performance applications.,0,19.595627
22,"I would rate the eworkflow extremely satisfied when there will be no eworkflow latency issues, no broken functionality in validation or processing milestone and new applications are fixed as soon as possible when the issue is reported.",1,9.650106
23,I believe some of my issues are not technically eWorkflow itself related. Frustration connecting to VPN and staying connected makes all system functionality trying at times. I am overall satisfied with eWorkflow. The improvements we need to make are on the business end in regards to improving functionality that would effect us...New Business\n,1,9.759661
24,"The slowness to improve drastically, being able to see dates on the cases set and maybe comments of each case instead of having to go into it each time.",1,10.237490
25,It is best if we have a quick process to open the screen through eflow. Sometimes I'm experiencing delays and connection issues.,1,10.255917
26,"The interface should be more user friendly, The ease of using the application is not that good.\nThere should be a way to save predefined task which are common while working in every case.\nQuick IGO function should be there in all the validation and processing milestone and also if there could be an option to close the case in one go just like it is when withdrawing the case.",1,10.683915


In [ ]:
df_distance.to_excel('.xlsx')

In [196]:
N_CLUSTERS=2
from scipy.spatial import distance_matrix
df_dist_matrix = pd.DataFrame(distance_matrix(topic_centroids,
                                              topic_centroids),
                              index=range(N_CLUSTERS),
                              columns=range(N_CLUSTERS))
df_dist_matrix

,0,1
0,0.000000,17.280416
1,17.280416,0.000000


#The below code is to check if topic assigned via Agglomerative clustering is right if we compare the distances using manually calculated eucledian distance which we are using to get top sentences

In [197]:
test=df.copy()

In [198]:
test.head()

,Data,topic_ac,topic_distance
0,"A new system? There are entirely too many clicks needed to accomplish anything. Especially as a processor, it takes almost as much time to navigate all the clicks in eWF as it does to take the work we're logging while doing it. Plus, mouse clicks mean potential carpal tunnel damage. Need to reduce the number of mouse clicks to accomplish tasks.",1,11.321112
1,All of the internet windows not closing out when a window opens to view case notes for a case. \nNot having to pull up a case a second time to have it complete updating the case status.,1,13.955352
2,allow sort of related cases by case create/open date.,1,14.534324
3,Already is! EWF works great for me,1,16.453131
4,"Analytics tab does not help pull real time details in excel file, making it cumbersome to analyze large data set.",1,12.121514


In [199]:
for x in range(len(topic_centroids)):
    Topic_dists=[]
    for row in test.index:
        X_row = tensor[row]
        topic_dist= euclidean(topic_centroids[x], X_row)
        Topic_dists.append(topic_dist)
    test[x]=Topic_dists
       
    

In [200]:
test[test['topic_ac']==1].head()

,Data,topic_ac,topic_distance,0,1
0,"A new system? There are entirely too many clicks needed to accomplish anything. Especially as a processor, it takes almost as much time to navigate all the clicks in eWF as it does to take the work we're logging while doing it. Plus, mouse clicks mean potential carpal tunnel damage. Need to reduce the number of mouse clicks to accomplish tasks.",1,11.321112,23.719162,11.321112
1,All of the internet windows not closing out when a window opens to view case notes for a case. \nNot having to pull up a case a second time to have it complete updating the case status.,1,13.955352,24.618113,13.955352
2,allow sort of related cases by case create/open date.,1,14.534324,22.740011,14.534324
3,Already is! EWF works great for me,1,16.453131,18.593269,16.453131
4,"Analytics tab does not help pull real time details in excel file, making it cumbersome to analyze large data set.",1,12.121514,23.814465,12.121514


In [201]:
test[test['topic_ac']==0].head()

,Data,topic_ac,topic_distance,0,1
5,better stability and performance,0,21.987509,21.987509,22.893482
6,can't be done,0,31.282381,31.282381,33.948853
9,Consistency.,0,21.967854,21.967854,30.933516
16,extremely satisfied,0,18.081373,18.081373,25.977949
25,I am already extremely satisfied,0,18.321226,18.321226,22.358320


In [110]:
type(topic_dist)

numpy.ndarray